In [4]:
# Imports
import pandas as pd
import numpy as np
import sklearn
from sklearn import pipeline, ensemble
import os


from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.model_selection import train_test_split
from sklearn import linear_model
#from skimage import data
#from skimage import transform
#from skimage import img_as_float
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsClassifier
import gzip
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import learning_curve

In [7]:
# File Reading and Splitting

# Read files
data = []
labels = []
longest_sample_length = 0

for root, dirpath, files in os.walk("Spectrograms"):
        for folder in dirpath:
            for r, d, files_in_folder in os.walk(os.path.join("Spectrograms", folder)):
                for f in files_in_folder:
                    path = os.path.join(root, folder, f)
                    d = np.load(path)
                    
                    
                    if len(d[1]) > longest_sample_length:
                        longest_sample_length = len(d[1])
                    
                    data.append(d)
                    labels.append(folder)
                    

data_new = []
for i in range(len(data)):
    d = data[i][1]
    num_samp = longest_sample_length // len(d)
    remainder = longest_sample_length % len(d)
    
    temp = []
    for j in range(num_samp):
        temp.extend(d)
    temp.extend(d[:remainder])
    
    data_new.append(temp)

print("Finished converting and padding data")

#dx = [data_new, labels]
#dx = np.swapaxes(dx, 0, 1)

#split2 = 10 / 90
#train_val, test = sklearn.model_selection.train_test_split(dx, test_size=0.10, random_state=42)
#train, val = sklearn.model_selection.train_test_split(train_val, test_size=split2)
pre_split = pd.DataFrame(data_new)
training_data, eval_data, training_labels, eval_labels = split(pre_split, labels, .2)
train_data, test_data, train_labels, test_labels = split(training_data, training_labels, .25)

print("Split Complete! Train Size:", train_data.shape, "Validation Size:", eval_data.shape, "Test Size:", test_data.shape)

Finished converting and padding data
Split Complete! Train Size: (1780, 173) Validation Size: (594, 173) Test Size: (594, 173)


In [3]:

#pre_split.head
print(pre_split.shape)

(2968, 173)


In [6]:
def split(data, labels, size):
    train_ndarray, test_ndarray, train_label_nd, test_label_nd = train_test_split(data, labels, test_size = size, random_state = 42)
    d_train = pd.DataFrame(train_ndarray)
    d_test = pd.DataFrame(test_ndarray)
    l_train = pd.DataFrame(train_label_nd)
    l_test = pd.DataFrame(test_label_nd)
    return d_train, d_test, train_label_nd, test_label_nd

In [7]:
# Rotate all on axes
train = np.swapaxes(train, 0, 1)
val = np.swapaxes(val, 0, 1)

NameError: name 'train' is not defined

In [4]:
tdf = pd.DataFrame()
vdf = pd.DataFrame()

for i in range(len(train[0])):
    tdf[i] = train[0][i]
for i in range(len(val[0])):
    vdf[i] = val[0][i]
    
tdf = np.swapaxes(tdf, 0, 1)
vdf = np.swapaxes(vdf, 0, 1)

tdf.head()

,0,1,2,3,4,5,6,7,8,9,...,86123,86124,86125,86126,86127,86128,86129,86130,86131,86132
0,14.626539,23.390087,19.426838,14.432089,0.898856,-14.858397,-27.177113,-37.953362,-31.702953,-38.369938,...,-2.219558,21.787386,36.240211,34.810318,27.006645,31.055195,25.510529,14.626539,23.390087,19.426838
1,-72.245804,-58.438854,-5.313781,1.269375,0.966032,1.326349,1.495742,2.079146,3.168158,1.657128,...,-135.508057,-139.348083,-136.562988,-139.832336,-136.977127,-140.374237,-138.512329,-143.202759,-140.010498,-143.028336
2,70.675308,51.611614,21.229004,-9.737505,-9.156812,-7.260016,-11.004324,-17.283920,-79.607071,-108.711838,...,-7.260016,-11.004324,-17.283920,-79.607071,-108.711838,-95.857849,-78.948616,-56.688427,-75.957779,-78.550049
3,-66.979813,-77.862198,-84.822495,-70.189697,-45.930717,-24.283543,-66.367439,-87.631554,-74.259201,-72.451195,...,-86.178391,-58.487511,-23.321863,-43.780670,-73.269028,-56.399788,-34.723007,-66.979813,-77.862198,-84.822495
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-18.285852,...,-1.160834,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-6.240053,-38.467331


In [ ]:
##pipeline = sklearn.pipeline.make_pipeline(sklearn.ensemble.RandomForestClassifier(n_estimators=2500, max_depth=None,random_state=0))
logistic_C_test = LogisticRegressionCV(random_state=0, max_iter=30000,Cs = 12).fit(train_data,train_labels)

print(logistic_C_test.score(test_data, test_labels))


/opt/conda/envs/common/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/conda/envs/common/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modu

In [ ]:
logistic_C_test = LogisticRegressionCV(random_state=0, max_iter=30000,Cs = 8).fit(train_data,train_labels)

print(logistic_C_test.score(test_data, test_labels))

In [ ]:
logistic_C_test = LogisticRegressionCV(random_state=0, max_iter=30000,Cs = 10).fit(train_data,train_labels)

print(logistic_C_test.score(test_data, test_labels))

In [ ]:
logistic_C_test = LogisticRegressionCV(random_state=0, max_iter=30000,Cs = 5).fit(train_data,train_labels)

print(logistic_C_test.score(test_data, test_labels))

In [ ]:
logistic_C_test = LogisticRegressionCV(random_state=0, max_iter=30000,Cs = 15).fit(train_data,train_labels)

print(logistic_C_test.score(test_data, test_labels))

In [ ]:
bagging_test = BaggingClassifier(n_estimators=100,random_state=0)
bagging_test.fit(train_data,train_labels)
print(bagging_test.score(test_data, test_labels))

In [ ]:
print(bagging_test.score(test_data, test_labels))

In [ ]:
train_sizes, train_scores, test_scores, fit_times, _ = learning_curve(bagging_test, test_data, test_labels,return_times=True)


In [ ]:
plt.figure()
plt.title("test")

plt.xlabel("Training examples")
plt.ylabel("Score")

train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)
plt.grid()

plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.1,
                 color="r")
plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.1, color="g")
plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
         label="Training score")
plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
         label="Cross-validation score")

plt.legend(loc="best")

In [9]:
GRB_mse_test = GradientBoostingClassifier(n_estimators=1000, max_depth=1, random_state=0)
GRB_mse_test.fit(train_data,train_labels)
print(GRB_mse_test.score(test_data, test_labels))

KeyboardInterrupt: 

In [ ]:
clf = sklearn.linear_model.SGDClassifier()
clf.fit(train_data,train_labels)

print(clf.score(test_data,test_labels))